Primero se me ocurre definir un código tal que nos ayude a realizar busqueda de valores dentro de un dataframe, dentro de una database o incluso dentro de una lista o numpy array, esta por verse, aunque se me ocurre que sería más fácil de realizar con Pandas y el método "loc".

In [69]:
import pandas as pd
data = {
    'ProbaFallecimiento': [0.5, 0.2, 0.3, 0.4, 0.5],
    'Edad': [25, 30, 28, 22, 35]
}

df = pd.DataFrame(data)

print(df)


probFallEdad = 25
valor = df.loc[df['Edad']  == probFallEdad,'ProbaFallecimiento']
print(type(valor))

   ProbaFallecimiento  Edad
0                 0.5    25
1                 0.2    30
2                 0.3    28
3                 0.4    22
4                 0.5    35
<class 'pandas.core.series.Series'>


Como podemos observar en este caso, el valor que esperamos es de tipo clase, lo cual no queremos, tendría que ser de tipo entero (int) o por lo menos alguna de las variantes que tiene Pandas pero que sea aún manipulable para hacer operaciones... (int64, etc)
Entonces debemos de revisar que esté nuestro dataframe (sus columnas) bajo un tipo de dato correcto, en este caso int para edades y float para Probas de fallecimiento

In [70]:

df['ProbaFallecimiento'] = df['ProbaFallecimiento'].astype(float)
df['Edad'] = df['Edad'].astype(int)
df.info()

probFallEdad = 25
valor = df.loc[df['Edad']  == probFallEdad,'ProbaFallecimiento'].item()
print(type(valor))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ProbaFallecimiento  5 non-null      float64
 1   Edad                5 non-null      int64  
dtypes: float64(1), int64(1)
memory usage: 208.0 bytes
<class 'float'>


¿Qué hace .item()?

Solo funciona si la Serie contiene exactamente un elemento
Extrae ese único valor y lo devuelve como un escalar puro (float o int)
Internamente es como hacer .values[0].item() en NumPy
Importante: Si la Serie tiene más de un valor, .item() lanza un error.

O bien, pudimos acceder al valor unicamente con la indexación en caso que loc me regresara en vez de un valor, valores, dentro de una SERIE, no lista, SERIE.
valor = df.loc[df['Edad'] == 25, 'ProbaFallecimiento'].values[0]

In [71]:
print(valor)


0.5


# Ejercicio de Seguro Dotal Mixto Realizado en Clase
Anteriormente habiamos realizado un ejercicio donde calculabamos la PUR y el valor de la Suma Asegurada en caso de sobrevivencia, en este caso para un dotal mixto EN EXCEL trataré de replicar ese ejercico dentro de Python, con los datos y calculos que hice y obtuve, los datos son los siguientes:
- Edad inicio (x)	    20
- Temporalidad (n)	    5 años
- Tasa de interés (i)	3.5% (0.035)
- SA Fallecimiento	    $50,000
- SA Sobrevivencia	    ? - $52,762.56

La tabla de mortalidad (que la transformaremos en un dataframe para este ejercico) es:
Edad  t_P_x  q_{x+t}
20	  0.01	  0.99
21	  0.015	  0.985
22	  0.02	  0.98
23	  0.025	  0.975
24	  0.03	  0.97

## Primer problema al que nos enfrentamos:
Veamos como vamos a obtener nuestra tabla principal que sería el calculo de *la probabilidad de que una persona de edad x+t sobreviva a edad x+t+1* y *la probabildiad de que una persona de edad x sobreviva a edad x+t*


In [72]:
# Primero creeamos nuestro dataframe manualmente, en este caso ÚNICAMENTE
# Recuerda que nos debemos de asegurar que esten en un tipo de deto corecto, en este caso edad en tipo int, q_x y p_x en float

data = {
    'edad': [20, 21, 22, 23, 24],  
    'q_x': [0.01, 0.015, 0.02, 0.025, 0.03],  
    'p_x': [0.99, 0.985, 0.98, 0.975, 0.97]  
}

tablaMort = pd.DataFrame(data)

# Cuidamos el tipo de dato de cada columna de nuestro DataFrame
tablaMort['edad'] = tablaMort['edad'].astype('int')
tablaMort['q_x'] = tablaMort['q_x'].astype('float')
tablaMort['p_x'] = tablaMort['p_x'].astype('float')

print(tablaMort.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   edad    5 non-null      int64  
 1   q_x     5 non-null      float64
 2   p_x     5 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 248.0 bytes
None


Ahora, lo que se me ocurre es primero adaptar los calculos a una sola edad, tal como viene planteado el problema, despues se me ocurre que podríamos cambiarlo por una variable y hacer de todo el codigo DINÁMICO, que es lo que siempre se busca y no tener un código estático.

In [73]:
# Declaro mis variables con valores fijos
edadContrata = 20

temporalidad = 5

tasa = 3.5

SAfall = 50000

Ahora trataré de resolver nuestra primer problemática, la creación de un dataframe para nuestros calculos

Se me ocurre, viendo la estructura de la creación de nuestro dataframe manual previo, partir de la base de este, los valores, los valores se definen a travez de listas, el problema es que sería 0 dinámico con el tiempo y tendría que definirse si se cambia de temporalidad valor por valor y que flojera.

QUIZÁ, una función, que trabaje con APPEND, para ir añadiendo valores, de forma recursiva quizá.

Creo que debería trabajar ademas con Numpy, porque usar una lista normal de Python para operar sería mala idea.


In [79]:
import numpy as np

# AQUÍ YA VA EL ANÁLISIS LINEA POR LINEA DE LO QUE DEBE DE SUCEDER PARA OBTENER LA PRIMER LISTA DE NUESTRA TABLA


valoresT = np.arange(temporalidad)
print(valoresT)

#Estas son las edades a las cuales accedería para mi flujo
edadesFLUJO = valoresT + edadContrata
print('Estas son las edades que debería de buscar para crear t_P_x', edadesFLUJO)
#Para acceder a cada uno de esos valores podría usar un ciclo for

# Veamos antes si sirve nuestro primer codigo de busqueda con loc, para este dataframe nuevo de este problema 
valorN = tablaMort.loc[tablaMort['edad']  == 20,'p_x'].item()
print('Tipo de dato = ', type(valorN))
print(valorN)

#Aquí contendré en una lista los valores que me interesan, osea mis probabilidades P_{x+t}
probabilidesP_xt = []
for i in edadesFLUJO:
    #Usaré mi primer código que busca los valores correspondientes dentro del dataframe
    probabilidesP_xt.append(tablaMort.loc[tablaMort['edad']  == i,'p_x'].item())

print(probabilidesP_xt)

# SIRVE

[0 1 2 3 4]
Estas son las edades que debería de buscar para crear t_P_x [20 21 22 23 24]
Tipo de dato =  <class 'float'>
0.99
[0.99, 0.985, 0.98, 0.975, 0.97]


Listo, tenemos por lo menos una columna creada, a decir verdad, la forma de solucionar este problema en Python se me hace muy muy muy muy muy diferente en cuanto a la forma de ver el problema, de interpretarlo, y claro, es codigo.

Ahora, pasemos a la siguiente columna, que usa resurividad.

En general, la resurvidad si o si se relaciona con funciones, pero no se hacerlas bien hasta el momento, todos los codigos de recursivdad que ví si se me complicaban inclusive códigos básicos como el de Fibonacci, donde se obtenian sus números, pero podemos preguntarle a chat que nos recuerde como era este tipo de codigo y adpatarlo a nuestra necesidad, no sería dificil porque es una multiplicación recursiva, por ende se me hace buena idea recurir al código de recursividad de Fibonacci, que era pura suma (creo).

CHATGPT:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

Ejemplo: imprimir los primeros 10 números de Fibonacci
for i in range(10):
    print(f"F({i}) = {fibonacci(i)}")

Neceistamos en nuestro caso, ver como se comporta el codigo paso a paso, como se comporta cuando toma n un valor, por ejemplo 5...
fibonacci(5) = fibonacci(4) + fibonacci(3)
fibonacci(4) = fibonacci(3) + fibonacci(2)
fibonacci(3) = fibonacci(2) + fibonacci(1)
fibonacci(2) = fibonacci(1) + fibonacci(0)

Hace un tipo de empaquetado de los valores 

Osea

fibonacci(CALCULA - 2) = 1 + 0 = 1 - Empaqueta y lleva al calculo de abajo - CASO BASE (pues los valores ya estan definidos)
fibonacci(CALCULA - 3) = fibonacci(2) + fibonacci(1) = 1 + 1 = 2 - Empaqueta y lo usa en el calculo de abajo
fibonacci(CALCULA - 4) = fibonacci(3) + fibonacci(2) = 2 + 1 = 3 - Empaqueta y lo usa en el calculo de abajo
fibonacci(CALCULA - 5) = fibonacci(4) + fibonacci(3) = 3 + 2 = 5 - Empaqueta y lo usa en el calculo de abajo


Ahora si lo adaptamos a lo que nosotros queremos es algo del tipo
funcion(CALCULA - 0_P_20) = 1 empaqueta  y lo lleva queremos usar en el calculo de abajo - CASO BASE, (pues el valor para este ya esta definido previamente)
funcion(CALCULA - 1_P_20) = funcion(CALCULA - 0_P_20) * función(1_P_20 - Tenemos y que tendría que BUSCAR) - **Segundo Problema** - EMPAQUETA
funcion(CALCULA - 2_P_20) = funcion(CALCULA - 1_P_20) * funcion(1_P_21 - Tenemos y que tendría que BUSCAR) - EMPAQUETA
...
Nota que para este caso llegariamos hasta  
funcion(CALCULA - 5_P_20) = funcion(CALCULA - 4_P_20) * funcion(1_P_24 - Tenemos y que tendría que BUSCAR)

*El codigo lo que hace es empezaar desde abajo respecto al ultimo análisis que hicimos, osea, parte de declarar que la temporalidad es 5 y de ahí se sigue desarrollando hasta llegar al valor t=0*
Esto debió de ser más que evidente para nuestro calculo

# Segundo Problema 
Ahora, el problema radica en tener en disposición los valores que necesito para que se operen dentro de la recursividad...
Los valores a buscar son
- 1_P_20
- 1_P_21
- 1_P_22
- 1_P_23
- 1_P_24

Que ya tenemos dentro de un array recuerda
Podriamos trabajar directamente sobre ese array, osea hacer alusión y uso directo de ese array donde estan las probabilidades contenidas en orden de esos valores que necesitamos

VARIABLE DEFINIDA COMO "probabilidesP_xt" contiene esas probabildiades en una lista

En una lista, por ende podemos trabajar sobre indexación (mi peor pesadilla de los algortimos de ordenamiento que se construían bajo indexación)

Osea lo que tratamos de hacer es:

funcion(0) = 1                          <- caso base
funcion(1) = funcion(0) * 0.99          = 1 * 0.99
funcion(2) = funcion(1) * 0.985         = 0.99 * 0.985
funcion(3) = funcion(2) * 0.98
funcion(4) = funcion(3) * 0.975
funcion(5) = funcion(4) * 0.97

La verdad, en este punto ya necesita un poco de orientación y le pase mi idea a chat y me dio la siguiente solución:
def calcular_probabilidad(t, probabilidadesP_xt):
    if t == 0:
        return 1
    else:
        return calcular_probabilidad(t - 1, probabilidadesP_xt) * probabilidadesP_xt[t - 1]



In [82]:
print(probabilidesP_xt)


def calcular_probabilidad(t, probabilidadesP_xt):
    if t == 0:
        return 1
    else:
        return calcular_probabilidad(t - 1, probabilidadesP_xt) * probabilidadesP_xt[t - 1]
    
resultado = calcular_probabilidad(5, probabilidesP_xt)

print(resultado)

[0.99, 0.985, 0.98, 0.975, 0.97]
0.9038031502499998


Lo peor es que si funciona y viendo el codigo si era claro que tenía que trabajar con indexación pero no se me hubiera ocurrido escribirlo de esa forma con respecto a la lista con las probabilidades.
Pero al menos ganamos experiencia con indices.

# Segundo Problema Solucionado
Ahora, ya tenemos la función que me retorna el valor de t_P_x

# Terecer Problema - FLUJOS CONTINGENTES 
Cada flujo trabaja sobre sus probabilidades y sobre su valor presente con respecto a su valor de t
Podriamos crear una función que trabaje sobre otras funciones (no se si eso es de buenas prácticas o si hay complejidad algorítmica pero asi de pensamiento, no veo problema con ello en ambos)

Recordemos que el flujo para t = 0 por ejemplo es 
SA * 0_P_20 * Q_20 * v^1

Donde ya tenemos...
- La suma asegurada al ser en este problema fija pues no hay problema, se declara como una variable, pero cuando este cambie PODEMOS DEFINIR UNA FUNCIÓN QUE SE MUEVA CON RESPECTO A T Y SU SUMA ASEGURADA
- Tenemos la función que nos calcula t_P_x (la última que hicimos)

Nos falta
- Se me ocurre definir una funcion para Q_20 para que sea este a su vez dinámica y se mueva entre los valores
POSTERIORMENTE, se me ocurre dentro de la clase que será nuestro calculador determinar errores con comprobación de valores. Pero esto ya mas adenlante si sale esto.
- Funcion para valor presente

**Posteriormente se me ocurre definir un ciclo for i in temporalidadesT tal que hará los flujos contingentes**

In [ ]:
# CODIGO DE LO QUE NOS FALTA

# Reutilizamos el codigo que hicimos al inicio y que busca los valores, en este caso queremos que busque sobre q_x con respecto a un valor t
t = 3

def Prob_Q_x(t):
    # OJO, que aqui trabajaremos con la edad de contratación en adición con t, pudimos trabajarlo de otra forma, claro, hay varias formas, por ejemplo, trabajar cn una lista que ya tuviera las edades sumadas, pero esta fue la primera forma que se me ocurrió
    buscar_xt = edadContrata + t
    valor = tablaMort.loc[tablaMort['edad']  == buscar_xt,'q_x'].item()
    return valor

print(Prob_Q_x(t))


0.025


Pero, se me ocurre que quizá los flujos podriamos hacerlos con numpy directamente y operar sobre numpy arrays...
Osea que nuestros datos se alacenen dentro de un numpy array PARA FALLECIMIENTO

Esta idea con el anterior codigo la desarrollaría de la siguiente forma


In [112]:
listaQ_x = []

for i in valoresT:
    listaQ_x.append(Prob_Q_x(i))

print(listaQ_x)

valoresQ_x = np.array(listaQ_x)

print(valoresQ_x)
print(type(valoresQ_x))

[0.01, 0.015, 0.02, 0.025, 0.03]
[0.01  0.015 0.02  0.025 0.03 ]
<class 'numpy.ndarray'>


Ahora la misma idea para el valor presente

In [105]:
listaVt = []

for i in valoresT:
    listaVt.append((1/(1+(tasa/100)))**i)

print(listaVt)

valoresVt = np.array(listaVt)
print(valoresVt)


[np.float64(1.0), np.float64(0.9661835748792271), np.float64(0.933510700366403), np.float64(0.9019427056680224), np.float64(0.8714422276985724)]
[1.         0.96618357 0.9335107  0.90194271 0.87144223]


Ahora la misma idea para los valores recursivos t_P_x

In [103]:
listat_P_x = []

for i in valoresT:
    listat_P_x.append(calcular_probabilidad(i, probabilidesP_xt))

print(listat_P_x)

valorest_P_x = np.array(listat_P_x)

print(valorest_P_x)

[1, 0.99, 0.97515, 0.9556469999999999, 0.9317558249999999]
[1.         0.99       0.97515    0.955647   0.93175582]


Ejemplos de como tendríamos los flujos calculados y el calculo de la prima unica de riesgo no es mas que la suma de esos flujos

In [ ]:
print(SAfall)
print(valorest_P_x)
print(valoresQ_x)
print(valoresVt)

flujoFall = SAfall * valorest_P_x * valoresQ_x * valoresVt

print(flujoFall)

print('PRIMA UNICA DE RIESGO PARA SEGURO TEMPORAL =', float(flujoFall.sum()))


50000
[1.         0.99       0.97515    0.955647   0.93175582]
[0.01  0.015 0.02  0.025 0.03 ]
[1.         0.96618357 0.9335107  0.90194271 0.87144223]
[ 500.          717.39130435  910.31295946 1077.42355105 1217.95705771]
PRIMA UNICA DE RIESGO = 4423.084872578216


Resolver  el problema de encontrar la suma asegurada para sobrevivencia ya es solo cuestión de aplicar nuestras funciones a los valores correspondientes, pero en teoría ya esta hecho lo que nos indetersaba y con ello podemos construir nuetro calculador de primas.

Lo ultimo refleja la paga de la prima unica si el seguro fuera unicamente temporal.
Para ser dotal mixto aplicamos la idea de arriba de usar nuestras funciones y sumarlas.

Esto dentro de una clase, sería un método calculador, que calcularía la prima UNICA, ya sea de un temporal, dotal puro o dotal mixto, practicamente con el codigo ya hecho sale, y es dinámico, asi que, va bien.